# Model Training Script
Lindsay Fitzpatrick
ljob@umich.edu
08/19/2024

This script reads in CFSR data from 1979 - 2010 and trains different machine learning
models to predict water levels across the 5 Great Lakes simultaeously.

In [37]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import WhiteKernel, ConstantKernel, RBF, Matern, RationalQuadratic
from sklearn.metrics import mean_squared_error, r2_score
import tensorflow as tf
from sklearn.ensemble import RandomForestRegressor
import joblib

User Input

In [2]:
# This is the directory where the CFSR and GLCC files are located
dir = 'C:/Users/fitzpatrick/Desktop/Data/Input/'

Read in the 3 CSV files with CFSR input data

In [3]:
## Read in PCP data from CFSR
data_1 = pd.read_csv(dir+'CFSR_APCP_Basin_Sums.csv',sep=',')

## Read in EVAP data from CFSR
data_2 = pd.read_csv(dir+'CFSR_EVAP_Basin_Sums.csv',sep=',')

## Read in TMP data from CFSR
data_3 = pd.read_csv(dir+'CFSR_TMP_Basin_Avgs.csv',sep=',')

Read in the GLCC RNBS CSV file

In [4]:
# Read in GLCC RNBS data
data_4 = pd.read_csv(dir + 'rnbs_glcc.csv', sep=',')

# Ensure 'Date' column is treated as string
date_strs = data_4['Date'].astype(str)

# The date in this file is a little goofy so we have to do some pre-processing
# to accurately read in the date as a datetime and align with other data sets
date_rnbs = []

for date_str in date_strs:
    # Insert leading zero before month
    if len(date_str) == 6:
        date_str = date_str[:4] + '0' + date_str[4:]

    # Insert leading zero before day
    if len(date_str) == 7:
        date_str = date_str[:6] + '0' + date_str[6:]

    # Convert to datetime object
    datetime_obj = datetime.strptime(date_str, '%Y%m%d')

    # Format the datetime object as a string in the desired format
    date_tmp = datetime_obj.strftime('%Y-%m-%d')

    # Append formatted date to list
    date_rnbs.append(date_tmp)

# Replace the original 'Date' column with formatted dates
data_4['Date'] = date_rnbs

# Convert 'Date' column to datetime format
data_4['Date'] = pd.to_datetime(data_4['Date'])

Here we prepare the data for training and testing. We set the features 'X' as total over lake
precipitation, total over lake evaporation, and the average air temperature over each lake. The
targets 'y' are RNBS for each lake simultaeously.

In [5]:
# Features
X = pd.DataFrame({
    'su_pcp': data_1['WaterSuperior'],
    'er_pcp': data_1['WaterErie'],
    'on_pcp': data_1['WaterOntario'],
    'mh_pcp': data_1['WaterMichigan']+data_1['WaterHuron'], # add the sums
    'su_evap': data_2['WaterSuperior'],
    'er_evap': data_2['WaterErie'],
    'on_evap': data_2['WaterOntario'],
    'mh_evap': data_2['WaterMichigan']+data_2['WaterHuron'], # add the sums
    'su_tmp': data_3['WaterSuperior'],
    'er_tmp': data_3['WaterErie'],
    'on_tmp': data_3['WaterOntario'],
    'mh_tmp': (data_3['WaterMichigan']+data_3['WaterHuron'])/2 # take the average temp
})

# Set the index by date
X.set_index(pd.to_datetime(data_1[['year', 'month']].assign(day=1)), inplace=True)

# Targets
rnbs = pd.DataFrame({
    'su_rnbs': data_4['sup'],
    'er_rnbs': data_4['eri'],
    'on_rnbs': data_4['ont'],
    'mh_rnbs': data_4['mic_hur']
})

# Set the index by date
rnbs.set_index(pd.to_datetime(data_4['Date']), inplace=True)

# Merge the features and targets to align with the dates
# Drops the dates in the RNBS where we don't have CFS data 
merged_df = pd.merge(X, rnbs, left_index=True, right_index=True, how='inner')

# Pull the target variables back out 
y = merged_df[['su_rnbs', 'er_rnbs', 'on_rnbs', 'mh_rnbs']]

Split the data into training and testing data sets. We could do it as a random 80/20 split
but instead we set split the data set by date ranges. This can easily be adjusted.

In [41]:
# Split data into training and testing sets
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
train_start_date = '1979-01-01'
train_end_date = '2004-12-01'
# Testing dataset
val_start_date = '2005-01-01'
val_end_date = '2011-01-01'

X_train = X[train_start_date:train_end_date]
y_train = y[train_start_date:train_end_date]
X_test = X[val_start_date:val_end_date]
y_test = y[val_start_date:val_end_date]

It is best practice to standardize the data from 0-1 before training

In [ ]:
# Standardize the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
y_train_scaled = scaler.fit_transform(y_train)
X_test_scaled = scaler.fit_transform(X_test)
y_test_scaled = scaler.fit_transform(y_test)

## Training
Below we train different models using the same data and calculate the r squared values on the 
test data to compare performance.

In [47]:
# Gaussian Process Regression
from sklearn.gaussian_process.kernels import RBF, ExpSineSquared, ConstantKernel
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Testing Different Kernels
# Basic kernel using ConstantKernel: r2 = 0.8259
#kernel = ConstantKernel(1.0, (1e-3, 1e3)) * RBF(1.0, (1e-2, 1e2))

# Matt's optimal kernel: r2 = 0.8315
kernel = 1.0 * Matern(nu=1.5) * RationalQuadratic()

# Test to add a seasonality component: r2 = 0.8279
#period = 3.0  # Period of the season
#kernel = ConstantKernel(1.0, (1e-3, 1e3)) * RBF(length_scale=1.0, length_scale_bounds=(1e-2, 1e2)) + ExpSineSquared(length_scale=1.0, periodicity=period, periodicity_bounds=(1e-2, 1e2))

# Set up the model
gpr = GaussianProcessRegressor(kernel=kernel, alpha=0.1, n_restarts_optimizer=10, random_state=42)

# Fit the model
gpr.fit(X_train_scaled, y_train_scaled)

# Save the trained model
joblib.dump(gpr, 'GP_trained_model.pkl')

# Predictions
y_pred, sigma = gpr.predict(X_test_scaled, return_std=True)

# Calculate Mean Squared Error
mse = mean_squared_error(y_test_scaled, y_pred)
r_squared = r2_score(y_test_scaled, y_pred)
print(f"The r squared value for the model is {r_squared}")
print(f"Mean Squared Error: {mse}")

The r squared value for the model is 0.8314745431157478
Mean Squared Error: 0.16852545688425236


In [48]:
## Random Forest Regressor Model: r2 = 0.7295

# Initialize RandomForestRegressor
model = RandomForestRegressor(random_state=42)

# Train the model
model.fit(X_train, y_train)

# Save the trained model
joblib.dump(model, 'RF_trained_model.pkl')

# Predict on the test set
y_pred = model.predict(X_test)

# Calculate Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
r_squared = r2_score(y_test, y_pred)
print(f"The r squared value for the model is {r_squared}")
print(f"Mean Squared Error: {mse}")

The r squared value for the model is 0.729492091633517
Mean Squared Error: 2353.407846198809


In [49]:
## Neural Network: r2 = 0.6906

# Standardize the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the neural network architecture
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(4)  # Number of targets
])

# Compile the model
model.compile(optimizer='adam', loss='mse')  # Using mean squared error (mse) as the loss function

# Fit the model to the training data
model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

# Save the trained model
model.save('NN_trained_model.keras')

# Predict on the test set
y_pred = model.predict(X_test_scaled)

# Calculate Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
r_squared = r2_score(y_test, y_pred)
print(f"The r squared value for the model is {r_squared}")
print(f"Mean Squared Error: {mse}")

Epoch 1/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 19969.9805 - val_loss: 17593.1914
Epoch 2/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 18626.7676 - val_loss: 17488.5039
Epoch 3/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 19216.4160 - val_loss: 17354.9395
Epoch 4/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 17491.2188 - val_loss: 17179.6641
Epoch 5/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 18273.5391 - val_loss: 16935.0723
Epoch 6/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 18753.4434 - val_loss: 16599.7598
Epoch 7/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 18637.1133 - val_loss: 16149.2734
Epoch 8/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 17180.3906 - val_loss: 15551.4639
Epoch 9/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 15862.7197 - val_loss: 14785.4707
Epoch 10/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 16249.0811 - val_loss: 13808.3799
Epoch 11/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 15911.6543 - val_loss: 12627.0186
Epoch 1